In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# 토크나이저 관련 경고 무시
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

사용 디바이스: cuda


In [3]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

voice_df = pd.read_csv('/content/drive/MyDrive/AI_Hackathon/data/voice_df_f.csv', sep='\t')

In [4]:
voice_df.isnull().sum()

Unnamed: 0    0
content       0
target        0
dtype: int64

In [15]:
from sklearn.utils import resample
import pandas as pd

# voice_df 데이터프레임에서 content와 target 컬럼 추출
X = voice_df['content']
y = voice_df['target']

# 클래스 0와 클래스 1을 분리
class_0 = voice_df[voice_df['target'] == 0]
class_1 = voice_df[voice_df['target'] == 1]

# 클래스 1을 2배 복제하여 오버샘플링
oversampled_class_1 = resample(class_1, replace=True, n_samples=len(class_1) * 2)

# 클래스 0와 클래스 1을 합쳐 오버샘플링된 데이터 생성
voice = pd.concat([class_0, oversampled_class_1])


In [16]:
class_0.sum()

Unnamed: 0                                              3107829
content       친애하다 판사 라는 관련 드라마무한도전 완전 폐지 돼다 없어지다 무한도전 자주 자다...
target                                                        0
dtype: object

In [10]:
class_0

,Unnamed: 0,content,target
1012,1012,친애하다 판사 라는 관련 드라마,0
1013,1013,무한도전 완전 폐지 돼다 없어지다 무한도전 자주 자다 챙기다,0
1014,1014,따다,0
1015,1015,지다 호랑 계단 올라오다 운동 대요 서다 운동,0
1016,1016,뭔가 효과,0
...,...,...,...
2686,2686,개월 개월 할부 해주다 카드 번호 고요 비밀번호 자리 땡땡 이제 가입 결제,0
2687,2687,년생 년생 이니까 년생 년생 할인 넣다 가입 쿠폰 해주다 강의 신청 가능하다 신청 ...,0
2688,2688,아이디 다빈 해주다 비밀번호 케이 지다 김표 느낌표 해주다 이메일 자주 쓰다,0
2689,2689,문자 보내다 인강 듣다 방법 시간표 보내다 교재 최대한 빨리 보내다 수업 주일 주일...,0


In [17]:
class_1.sum()

Unnamed: 0                                               511566
content       은행권 들어가다 나오다 태도 갑자기 이렇다 본인 관할 경찰서 기범 여보세요 피해자 ...
target                                                     1012
dtype: object

In [18]:
voice.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [19]:
df_shuffled = sklearn.utils.shuffle(voice)

train, test = train_test_split(voice, test_size=0.2)

In [20]:
train

,content,target
2618,땡땡 학습 담당자 지연 감사하다,0
443,연말 니깐 피해자 알다 기범 부분 자다 말씀 해주시 정상 처리 년생 이시고 서울 거...,1
568,말씀 주다 말씀 드리다 공문서 자체 러시 캐시 고객 받다 상환 처리 이자 비용 거래...,1
1859,확실하다 진짜 많다,0
179,본인 카카오 톡 번만 들어가다 카카오 톡 한번 들어가다 피해자 기범 친구 목록 보시...,1
...,...,...
2348,회수 확인 등록 무조건 홈페이지 가다 등록 지도 궁금하다 전화 등록 나중 까먹다,0
180,조사 대신 참석 참작 드릴 전산 서다 보내다 기범 본인 카카오 톡 본인 성함 번만 ...,1
516,오정태 상담사 기범 상담 대출 상담 사기 현재 농협 나중 신한 은행 수도 국민은행 ...,1
1137,거기,0


In [21]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [22]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.0 MB/s eta 0:00:00


In [23]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader

In [24]:
class TokenDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        self.data = dataframe
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['content']
        label = self.data.iloc[idx]['target']

        # 토큰화 처리
        tokens = self.tokenizer(
            text,
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


In [25]:
# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME

# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=0)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
train_loader

In [ ]:
test_loader

In [ ]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [26]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

tensor([0, 1, 1, 0, 1, 1, 1, 0], device='cuda:0')

In [27]:
# 생성된 inputs의 key 값 출력
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [28]:
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([8, 512]), torch.Size([8, 512]), torch.Size([8, 512]))

In [29]:
from transformers import BertConfig

config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [30]:
# labels 출력
labels

tensor([0, 1, 1, 0, 1, 1, 1, 0])

In [31]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [32]:
output = model_bert(**inputs)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [33]:
output['last_hidden_state'].shape, output['pooler_output'].shape

(torch.Size([8, 512, 768]), torch.Size([8, 768]))

In [34]:
# last_hidden_state 출력
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

torch.Size([8, 512, 768])
tensor([[-1.1897e-01, -1.6132e-01, -1.7518e-01,  ..., -1.5286e-01,
         -1.1394e+00,  5.9737e-01],
        [-1.0956e-01, -5.2427e-01, -7.1866e-01,  ..., -5.2409e-01,
          8.6108e-01, -5.4943e-01],
        [-2.4628e-01, -6.1225e-01, -1.2968e+00,  ..., -7.4665e-01,
          3.3982e-01, -2.2106e-01],
        ...,
        [-1.2280e-03,  1.6572e-01, -6.2267e-01,  ..., -6.4186e-01,
          9.7126e-01, -2.7362e-01],
        [-2.8990e-01,  6.4787e-02, -7.3684e-01,  ..., -1.2418e-01,
          1.0683e+00, -1.6905e-01],
        [-6.7884e-01,  1.8255e-01, -3.3301e-01,  ..., -1.2082e+00,
         -8.9264e-01, -2.0855e-02]], device='cuda:0', grad_fn=<SliceBackward0>)


In [35]:
# pooler_output 출력
pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

torch.Size([8, 768])
tensor([[ 0.0780,  0.7724, -0.5598,  ...,  0.5999,  0.3745,  0.7246],
        [ 0.9248,  0.3214, -0.6872,  ..., -0.9147,  0.3765, -0.2308],
        [ 0.9228,  0.1926, -0.3524,  ..., -0.9439,  0.2130, -0.5512],
        ...,
        [ 0.8898,  0.4877, -0.9731,  ..., -0.4485,  0.0704, -0.0618],
        [ 0.9719,  0.3037, -0.0952,  ..., -0.9636,  0.3590, -0.1992],
        [ 0.9565, -0.0952, -0.9948,  ...,  0.4766,  0.6308, -0.1091]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [36]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

torch.Size([8, 2])
tensor([1, 1, 1, 1, 1, 0, 1, 0], device='cuda:0')


In [37]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [38]:
# CustomBertModel 생성
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

CustomBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [39]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [40]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [41]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)

            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [42]:
import numpy as np

# 최대 Epoch을 지정합니다.
num_epochs = 3

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 0.10285, training accuracy: 0.99662: 100%|██████████| 371/371 [04:30<00:00,  1.37batch/s]


[INFO] val_loss has been improved from inf to 0.00007. Saving Model!
epoch 01, loss: 0.01288, acc: 0.99662, val_loss: 0.00007, val_accuracy: 1.00000


training loss: 0.00153, training accuracy: 1.00000: 100%|██████████| 371/371 [04:32<00:00,  1.36batch/s]


[INFO] val_loss has been improved from 0.00007 to 0.00002. Saving Model!
epoch 02, loss: 0.00019, acc: 1.00000, val_loss: 0.00002, val_accuracy: 1.00000


training loss: 0.00063, training accuracy: 1.00000: 100%|██████████| 371/371 [04:32<00:00,  1.36batch/s]


[INFO] val_loss has been improved from 0.00002 to 0.00001. Saving Model!
epoch 03, loss: 0.00008, acc: 1.00000, val_loss: 0.00001, val_accuracy: 1.00000


In [43]:
# 저장한 state_dict를 로드 합니다.
bert.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

In [44]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [45]:
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '일반 대화 입니다.',
    1: '보이스피싱 입니다.'
}

# CustomPredictor 인스턴스를 생성
predictor = CustomPredictor(bert, tokenizer, labels)

In [46]:
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

In [53]:
# 예측할 대화 입력
predict_sentence(predictor)

문장을 입력해 주세요: 여보세요 네 안녕하세요 서울중앙지방검찰청 00부 담당 형사 000입니다. 000님 맞으십니까? 맞는데요 다름이 아니고 혹시 김00님을 아십니까? 지금 김00님이 경남은행에서 000님 명의로 대포통장을 개설하였습니다. 예? 지금 피해사례가 220건이 넘고 대포통장 개설해준 명목으로 000님께서 명의를 도용당하신 것인지, 범죄에 가담한 것인지 확인하려고 전화드렸는데요. 네?? 대포통장이요? 저 그런 적 없는데요 원래는 사건 참고인 조사는 방문조사 하시라고 우편으로 내용을 보내드리는데 긴급한 사항이고 방문하기 어려운 상황이시다 하시면 전화로 조사가 이루어집니다. 그래서 이렇게 유선으로 참고인조사가 이루어지고 있고요. 아 예.. 네 그럼 일단 대포통장 범죄에 000님이 가담하셨는지 아니면 정말 피해자이신건지 몇가지 사항으로 확인드리겠습니다. 괜찮으십니까? 네 그래주세요. 그 전에 본인확인 할텐데요. 녹취가 되고요. 정확하게 진술해주십시요.
[보이스피싱 입니다.]
확률은: 99.893% 입니다.
